# Query Registry Users

In [1]:
# set lifemonitor root path
lifemonitor_root = "/home/simleo/git/life_monitor"
%cd -q {lifemonitor_root}

In [2]:
import requests

In [3]:
# LifeMonitor URLs. Change port to 8443 for a production deployment
lm_base_url = "https://lm:8000"
lm_token_url = f"{lm_base_url}/oauth2/token"

In [4]:
# Copy the values of CLIENT_ID and CLIENT_SECRET from the output of "flask registry show seek"
CLIENT_ID = "g2ioTF0AzjFopXbRpSqPc2d2"
CLIENT_SECRET = "PFYk1MUypSVgczX3cfncPmvfNARJnUe9TUJD23JPgaAyxnXL"

In [5]:
s = requests.session()
s.verify = False
s.headers.update({})
token_response = s.post(
    lm_token_url, 
    data={
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "grant_type": "client_credentials",
        "scope": "read write"
    }, allow_redirects=True, verify=False)
assert token_response.status_code == 200
token = token_response.json()
token

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'access_token': 'AvgpfGYoeGjGra9Me7PB5JkDYR527RNYwaXpQrKvJF',
 'expires_in': 864000,
 'scope': 'read write',
 'token_type': 'Bearer'}

In [6]:
# Update headers with the OAuth2 token
s.headers.update({'Authorization': f"Bearer {token['access_token']}"})

In [7]:
# Enter the following URL in your browser
f"{lm_base_url}/oauth2/login/seek"
# Then log in as username: user1 and password: workflowhub

'https://lm:8000/oauth2/login/seek'

In [8]:
# Get registry users
response = s.get(f"{lm_base_url}/users")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_users = response.json()
registry_users

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': [{'id': 2,
   'identity': {'provider': {'name': 'seek',
     'type': 'seek',
     'uri': 'https://seek:3000',
     'userinfo_endpoint': 'https://seek:3000/people/current?format=json'},
    'sub': '2'},
   'username': 'qgkpNNXtqY'}]}